# Create Study

Creates a new study record in the `md_study` table and a protocol draft record in `md_protocol_draft`.

## Parameters
- `catalog`: Catalog name
- `study_id`: Unique study identifier
- `study_title`: Study title
- `study_description`: Study description (optional)
- `protocol_id`: Protocol document ID
- `created_by`: User who created the study

## Output Tables
- `gold_md.md_study` - Study registry
- `silver_md.md_protocol_draft` - Protocol draft records


In [ ]:
# Widget parameters
dbutils.widgets.text("catalog", "dta_poc_test", "Catalog")
dbutils.widgets.text("study_id", "", "Study ID")
dbutils.widgets.text("study_title", "", "Study Title")
dbutils.widgets.text("study_description", "", "Study Description")
dbutils.widgets.text("protocol_id", "", "Protocol ID")
dbutils.widgets.text("created_by", "", "Created By")

# Get parameter values
catalog = dbutils.widgets.get("catalog")
study_id = dbutils.widgets.get("study_id")
study_title = dbutils.widgets.get("study_title")
study_description = dbutils.widgets.get("study_description")
protocol_id = dbutils.widgets.get("protocol_id")
created_by = dbutils.widgets.get("created_by")

print(f"Catalog: {catalog}")
print(f"Study ID: {study_id}")
print(f"Study Title: {study_title}")
print(f"Protocol ID: {protocol_id}")
print(f"Created By: {created_by}")


In [ ]:
# Validate required parameters
if not study_id:
    raise ValueError("study_id is required")
if not study_title:
    raise ValueError("study_title is required")

# Define schema and table
SCHEMA = "gold_md"
TABLE = "md_study"
full_table_name = f"{catalog}.{SCHEMA}.{TABLE}"
print(f"Target table: {full_table_name}")


In [ ]:
# Create table if not exists
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {full_table_name} (
        study_id STRING NOT NULL COMMENT 'Unique study identifier',
        study_title STRING COMMENT 'Study title/name',
        study_description STRING COMMENT 'Study description',
        status STRING COMMENT 'Study status: IN_PROGRESS, PARSING_IN_PROGRESS, PARSING_COMPLETED, PARSING_FAILED, COMPLETED',
        protocol_id STRING COMMENT 'Associated protocol document ID',
        created_by_principal STRING COMMENT 'User who created the study',
        created_ts TIMESTAMP COMMENT 'Creation timestamp',
        last_updated_by_principal STRING COMMENT 'User who last updated the study',
        last_updated_ts TIMESTAMP COMMENT 'Last update timestamp'
    ) USING DELTA
    COMMENT 'Study registry for clinical data management'
""")
print(f"Table {full_table_name} ready")


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType
from datetime import datetime

# Escape single quotes in strings for SQL
study_title_escaped = study_title.replace("'", "''") if study_title else ""
study_description_escaped = study_description.replace("'", "''") if study_description else ""

# Check if study already exists
existing = spark.sql(f"SELECT study_id FROM {full_table_name} WHERE study_id = '{study_id}'")
if existing.count() > 0:
    print(f"Study {study_id} already exists - updating...")
    spark.sql(f"""
        UPDATE {full_table_name}
        SET study_title = '{study_title_escaped}',
            study_description = '{study_description_escaped}',
            last_updated_by_principal = '{created_by}',
            last_updated_ts = current_timestamp()
        WHERE study_id = '{study_id}'
    """)
    print(f"Study {study_id} updated successfully")
else:
    print(f"Creating new study: {study_id}")
    current_ts = datetime.now()
    
    # Create DataFrame with new study
    study_data = [(
        study_id,
        study_title,
        study_description or None,
        "IN_PROGRESS",  # Initial status
        protocol_id if protocol_id else None,  # Use protocol_id from parameter
        created_by or None,
        current_ts,
        created_by or None,
        current_ts
    )]
    
    schema = StructType([
        StructField("study_id", StringType(), False),
        StructField("study_title", StringType(), True),
        StructField("study_description", StringType(), True),
        StructField("status", StringType(), True),
        StructField("protocol_id", StringType(), True),
        StructField("created_by_principal", StringType(), True),
        StructField("created_ts", TimestampType(), True),
        StructField("last_updated_by_principal", StringType(), True),
        StructField("last_updated_ts", TimestampType(), True)
    ])
    
    df = spark.createDataFrame(study_data, schema)
    df.write.mode("append").saveAsTable(full_table_name)
    print(f"Study {study_id} created successfully")


In [ ]:
# ============================================================================
# Create md_protocol_draft record in Silver schema
# ============================================================================

SILVER_SCHEMA = "silver_md"
PROTOCOL_TABLE = "md_protocol_draft"
protocol_table_full = f"{catalog}.{SILVER_SCHEMA}.{PROTOCOL_TABLE}"

print(f"\n{'='*60}")
print(f"Creating Protocol Draft Record")
print(f"{'='*60}")
print(f"Target table: {protocol_table_full}")

# Create table if not exists
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {protocol_table_full} (
        protocol_id STRING NOT NULL COMMENT 'Unique protocol identifier',
        study_id STRING NOT NULL COMMENT 'Associated study ID',
        document_id STRING COMMENT 'Document ID from md_file_history',
        protocol_title STRING COMMENT 'Protocol title (from study)',
        protocol_version STRING COMMENT 'Protocol version',
        status STRING COMMENT 'Status: PENDING, PARSING, SECTIONS_EXTRACTED, FAILED',
        section_count INT COMMENT 'Number of sections extracted',
        created_by_principal STRING COMMENT 'User who created the record',
        created_ts TIMESTAMP COMMENT 'Creation timestamp',
        last_updated_by_principal STRING COMMENT 'User who last updated',
        last_updated_ts TIMESTAMP COMMENT 'Last update timestamp'
    ) USING DELTA
    COMMENT 'Protocol draft records linking studies to protocol documents'
""")
print(f"Table {protocol_table_full} ready")

# Only create protocol record if protocol_id is provided
if protocol_id:
    # Check if protocol already exists
    existing_protocol = spark.sql(f"SELECT protocol_id FROM {protocol_table_full} WHERE protocol_id = '{protocol_id}'")
    
    if existing_protocol.count() > 0:
        print(f"Protocol {protocol_id} already exists - updating status to PENDING...")
        spark.sql(f"""
            UPDATE {protocol_table_full}
            SET status = 'PENDING',
                last_updated_by_principal = '{created_by}',
                last_updated_ts = current_timestamp()
            WHERE protocol_id = '{protocol_id}'
        """)
    else:
        print(f"Creating new protocol draft: {protocol_id}")
        current_ts = datetime.now()
        
        protocol_data = [(
            protocol_id,
            study_id,
            None,  # document_id - will be set by file processor
            study_title,
            "1.0",  # Initial version
            "PENDING",  # Initial status
            0,  # section_count - will be updated after parsing
            created_by or None,
            current_ts,
            created_by or None,
            current_ts
        )]
        
        protocol_schema = StructType([
            StructField("protocol_id", StringType(), False),
            StructField("study_id", StringType(), False),
            StructField("document_id", StringType(), True),
            StructField("protocol_title", StringType(), True),
            StructField("protocol_version", StringType(), True),
            StructField("status", StringType(), True),
            StructField("section_count", IntegerType(), True),
            StructField("created_by_principal", StringType(), True),
            StructField("created_ts", TimestampType(), True),
            StructField("last_updated_by_principal", StringType(), True),
            StructField("last_updated_ts", TimestampType(), True)
        ])
        
        df_protocol = spark.createDataFrame(protocol_data, protocol_schema)
        df_protocol.write.mode("append").saveAsTable(protocol_table_full)
        print(f"Protocol draft {protocol_id} created successfully")
    
    # Display the created/updated protocol
    result_protocol = spark.sql(f"SELECT * FROM {protocol_table_full} WHERE protocol_id = '{protocol_id}'")
    display(result_protocol)
else:
    print("No protocol_id provided - skipping protocol draft creation")


In [ ]:
# Verify and display the created/updated study
result = spark.sql(f"SELECT * FROM {full_table_name} WHERE study_id = '{study_id}'")
display(result)

# Set output for downstream tasks
dbutils.notebook.exit(study_id)
